In [1]:
### Imports
import pandas as pd
import unidecode
import re
import math
import locale

### Users cleaning

In [6]:
# Load dataset
users = pd.read_csv('data/users.csv')

# To remove every non-ASCII character
users = users.applymap(lambda x: unidecode.unidecode(x) if isinstance(x, str) else x)

# Convert registration date string to proper year
users['registration_date'] = users['registration_date'].str.replace('Membre Tripadvisor depuis ', '')
users['registration_date'] = users['registration_date'].str.replace('None', '-1')
users['registration_date'] = users['registration_date'].astype(int)

# Convert number-of-comments string int an int value
def format_error(row):
    if 'ville' in row['nb_of_comments']:
        row['nb_of_cities'] = row['nb_of_comments']
        row['nb_of_comments'] = '-1'
    if 'vote' in row['nb_of_cities']:
        row['nb_of_comments'] = row['nb_of_cities']
        row['nb_of_cities'] = '-1'
    if 'vote' in row['nb_of_comments']:
        row['nb_of_comments'] = row['nb_of_comments'].replace(' votes utiles', '')
        row['nb_of_comments'] = row['nb_of_comments'].replace(' vote utile', '')
    if 'photo' in row['nb_of_cities']:
        row['nb_of_cities'] = '-1'
        
    return row

users['nb_of_comments'] = users['nb_of_comments'].str.replace('contributions', '')
users['nb_of_comments'] = users['nb_of_comments'].str.replace('contribution', '')
users['nb_of_comments'] = users['nb_of_comments'].str.replace('None', '-1')
users['nb_of_comments'] = users['nb_of_comments'].apply(unidecode.unidecode)
users = users.apply(format_error, axis=1)
users['nb_of_comments'] = users['nb_of_comments'].str.replace(' ', '')
users['nb_of_comments']= users['nb_of_comments'].astype(int)

# Convert the number-of-cities string into an int value
users['nb_of_cities'] = users['nb_of_cities'].str.replace('ville visitee', '')
users['nb_of_cities'] = users['nb_of_cities'].str.replace('villes visitees' ,'')
users['nb_of_cities'] = users['nb_of_cities'].str.replace('None', '-1')
users['nb_of_cities'] = users['nb_of_cities'].str.replace(' ', '')
users['nb_of_cities'] = users['nb_of_cities'].astype(int)

users['sex'] = users['sex'].str.replace('homme', 'H')
users['sex'] = users['sex'].str.replace('femme', 'F')

def format_cuisine_occur(cuisines):    
    if isinstance(cuisines, float) and math.isnan(cuisines):
        return {}
    
    cuisines_dict = {}
    pattern = '\d+ +(\w+) +(\d+)'
    matches = re.findall(pattern, cuisines)   
    
    for match in matches:
        if match[0] != 'None':
            cuisines_dict[match[0]] = int(match[1])
    return cuisines_dict

users['top_cuisines'] = users['top_cuisines'].apply(format_cuisine_occur)

users = users[['username', 'registration_date', 'nb_of_comments', 'nb_of_cities', 'age', 'sex', 'country', 'top_cuisines', 'avg_overall_rating', 'avg_cuisine_rating', 'avg_service_rating', 'avg_quality_price_rating', 'avg_mood_rating', 'avg_michelin', 'avg_price_range_estimation']]
users

,username,registration_date,nb_of_comments,nb_of_cities,age,sex,country,top_cuisines,avg_overall_rating,avg_cuisine_rating,avg_service_rating,avg_quality_price_rating,avg_mood_rating,avg_michelin,avg_price_range_estimation
0,nathalieval1972,2021,10,2,None,None,None,"{'Restauration_rapide': 3, 'Francaise': 2, 'Eu...",3.64,2.74,2.67,2.73,0.57,0.0,1.31
1,johanm8,2018,3,3,None,None,None,{},4.50,0.00,0.00,0.00,0.00,0.0,0.00
2,DumasDD,2013,23,14,50-64,H,france,"{'Europeenne': 2, 'Suisse': 1, 'Francaise': 1,...",4.17,2.67,2.50,2.50,1.33,0.0,1.00
3,ernestbenoit74,2016,35,22,None,None,france,"{'Mediterraneenne': 2, 'Italienne': 2, 'Europe...",4.00,2.05,2.05,1.78,1.50,0.0,1.30
4,cerisecherry,2011,64,40,35-49,F,france,"{'Francaise': 10, 'Europeenne': 10, 'Italienne...",4.11,3.90,3.84,3.63,2.18,0.0,1.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349990,Les_beaux_moments,2022,4,3,None,None,suisse,"{'Suisse': 1, 'Francaise': 1}",4.00,0.00,0.00,0.00,0.00,0.0,2.00
349991,Suifran,2019,1,1,None,None,None,"{'Suisse': 1, 'Francaise': 1}",4.00,0.00,0.00,0.00,0.00,0.0,2.00
349992,Peutmieux,2020,1,1,None,None,suisse,"{'Suisse': 1, 'Francaise': 1}",4.00,0.00,0.00,0.00,0.00,0.0,2.00
349993,Getaway00791979062,2019,1,1,None,None,france,"{'Suisse': 1, 'Francaise': 1}",4.00,0.00,0.00,0.00,0.00,0.0,2.00


### Restaurants cleaning

In [5]:
restaurants = pd.read_csv('data/restaurants.csv')
restaurants = restaurants.applymap(lambda x: unidecode.unidecode(x) if isinstance(x, str) else x)

relevant_meals = ['Diner', 'Petit dejeuner', 'Dejeuner']

def format_food_details(data, relevant, irrelevant):
    food = data.split(', ')
    filtered_food = [x for x in food if x not in irrelevant]
    if len(relevant) != 0:
        filtered_food = list(set(food).intersection(relevant))
    return filtered_food

restaurants['meal_types'] = restaurants['meal_types'].apply(lambda x : format_food_details(x, relevant_meals, []))
restaurants['cuisines'] = restaurants['cuisines'].apply(lambda x : format_food_details(x, [], ['0', 'None']))
restaurants['veggie'] = restaurants['special_regimes'].str.contains("Vegetariens")
restaurants = restaurants.rename(columns={'restaurant_name': 'name'})
restaurants = restaurants[['name', 'adress', 'city_ranking', 'overall_rating', 'number_of_reviews', 'price_range_estimation', 'veggie', 'meal_types', 'cuisines', 'cuisine_rating', 'service_rating', 'quality_price_rating', 'mood_rating', 'michelin']]
restaurants

,name,adress,city_ranking,overall_rating,number_of_reviews,price_range_estimation,veggie,meal_types,cuisines,cuisine_rating,service_rating,quality_price_rating,mood_rating,michelin
0,Bistro Spatz,"Dorfstrasse 2, Freienstein-Teufen 8427 Suisse",7.0,5.0,12,2,False,"[Dejeuner, Diner, Petit dejeuner]","[Suisse, Europeenne]",5.0,0.0,4.0,0.0,0
1,Wirtshus zum Wyberg,"Oberteufenerstrasse 1, Freienstein-Teufen 8428...",2.0,4.5,66,3,True,"[Dejeuner, Diner]","[Suisse, Europeenne, Europe centrale]",4.5,4.5,4.0,4.0,1
2,Alpengasthof Tibethuette,"Stilfser Joch, 39020, Stelvio Italie",33.0,4.5,3,0,False,[],[Italie du Nord],0.0,0.0,0.0,0.0,0
3,Tibet Hutte,"Passo Stelvio, 39029, Stelvio Italie",21.0,4.0,109,2,True,"[Dejeuner, Diner, Petit dejeuner]","[Italienne, Bar, Europeenne, Pub]",4.0,4.0,3.5,4.5,0
4,Restaurant Gallia,"Gomagoi 18, 39029, Stelvio Italie",18.0,4.5,79,2,True,"[Dejeuner, Diner, Petit dejeuner]","[Internationale, Italienne, Mediterraneenne]",5.0,4.5,4.5,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31638,Golden Dreieck,"Markthallenstr. 1, 78315 Radolfzell am Bodense...",131.0,3.5,20,1,True,[Dejeuner],"[Chinoise, Asiatique]",3.5,3.5,4.0,0.0,0
31639,Asia Food,"Robert-Gerwig-Str. 12, 78315 Radolfzell am Bod...",211.0,3.0,18,1,True,[Diner],[Asiatique],3.5,3.0,4.0,0.0,0
31640,Cobblers Cafe,"Schmidtengasse 9, 78315 Radolfzell am Bodensee...",121.0,4.5,9,2,False,[],[Cafe],5.0,5.0,5.0,0.0,0
31641,Indien Mirchi,"Bismarckstr. 2, 78315 Radolfzell am Bodensee, ...",63.0,4.0,90,2,True,"[Dejeuner, Diner]",[Indienne],4.0,4.5,4.0,0.0,0


### Reviews cleaning

In [4]:
reviews = pd.read_csv('data/reviews.csv')
def format_username(username):
    if not ', ' in username:
        return username
    words = re.findall('[A-Za-z0-9][^A-Z]*', username)
    return ''.join(words[:-2])

reviews['username'] = reviews['username'].apply(format_username)
locale.setlocale(locale.LC_TIME, 'fr_FR')
reviews['visit_date'] = reviews['visit_date'].str.replace('None', 'janvier 1900')
reviews['review_date'] = reviews['review_date'].str.replace('None', 'janvier 1900')

reviews['review_date'] = pd.to_datetime(reviews['review_date'], format='%d %B %Y')
reviews['visit_date'] = pd.to_datetime(reviews['visit_date'], format='%B %Y')
reviews = reviews[['restaurant_name', 'username', 'rating', 'review_date', 'visit_date']]
reviews.head()

,restaurant_name,username,rating,review_date,visit_date
0,Bistro Spatz,maximiuliana,5,2021-08-19,2021-07-01
1,Bistro Spatz,gedeonberger,5,2022-07-10,2022-07-01
2,Bistro Spatz,Ritschi64,5,2022-06-14,2022-06-01
3,Bistro Spatz,DomeH20,5,2022-02-04,2022-02-01
4,Bistro Spatz,Ritschi64,5,2021-11-03,2021-11-01
